# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try: 
    session.execute("""
                        CREATE KEYSPACE IF NOT EXISTS sparkify
                        WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};
                    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')

except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_session"
query += "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, "\
         "PRIMARY KEY((session_id, item_in_session)))" 

try: 
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
"""
This quey will provide the name of the artist, its song and the length of it for the session with id equal to 338 and
specificly the fourth item of the session

Primary key: has a compound partition key based on the fact that it needs to be unique and just with the "session_id",
             we can not guarantee this; hence, we add the "item_in_session" to reach this desired condition.
"""
query = "SELECT  artist_name, song_title, song_length FROM song_info_by_session WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print('Artist name: ', row.artist_name, ' | Song title: ', row.song_title, ' | Song length: ', row.song_length)

Artist name:  Faithless  | Song title:  Music Matters (Mark Knight Dub)  | Song length:  495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_id"
query += "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_full_name text,"\
          "PRIMARY KEY((user_id, session_id), item_in_session))"

try: 
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_id (user_id, session_id, item_in_session, artist_name, song_title, user_full_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + ' ' + line[4]))

"""
This quey will provide for a given "session_id" and "user_id" the name of the all the artists and songs; he/she has listen
to.

Primary key: has a compound partition key based on the fact that it needs to be unique and just with the "user_id",
             we can not guarantee this; hence, we add the "session_id" to reach this desired condition. Plus there is a
             clustering column "item in session", to obtain results sorted in ascending order by this attribute.
"""        
query = "SELECT  artist_name, song_title, user_full_name FROM song_info_by_id WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print('Artist name: ', row.artist_name, ' | Song title: ', row.song_title, ' | User name: ', row.user_full_name)

Artist name:  Down To The Bone  | Song title:  Keep On Keepin' On  | User name:  Sylvie Cruz
Artist name:  Three Drives  | Song title:  Greece 2000  | User name:  Sylvie Cruz
Artist name:  Sebastien Tellier  | Song title:  Kilometer  | User name:  Sylvie Cruz
Artist name:  Lonnie Gordon  | Song title:  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  | User name:  Sylvie Cruz


In [12]:
query = "CREATE TABLE IF NOT EXISTS music_library_by_song"
query += "(song_title text, user_id int, user_full_name text, PRIMARY KEY((song_title, user_id)))"

try: 
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_song (song_title, user_id, user_full_name)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1] + ' ' + line[4]))

"""
The goal of this query is to obtain the name of all of the users, that have listened to a specific song.

Primary key: in this case first attribute composing the composed node key, is the name of the song and the second one is
             the name of the user; having unicity guaranteed.
"""
query = "SELECT  user_full_name FROM music_library_by_song WHERE song_title='All Hands Against His Own' ALLOW FILTERING"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print('User name: ', row.user_full_name)

User name:  Tegan Levine
User name:  Sara Johnson
User name:  Jacqueline Lynch


### Drop the tables before closing out the sessions

In [13]:
drop_tables = ['song_info_by_session', 'song_info_by_id', 'music_library_by_song']

for drop_table in drop_tables:
    session.execute('DROP TABLE ' + drop_table)

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()